# Comparing Transect Data at the Tanana River Test Site 

The following example will familiarize the user with the MHKiT DOLfYN and Delft3D modules by performing a Deflt3d numerical model validation of the Tanana River Test Site against field data.

Start by importing the necessary python packages and MHKiT module.

In [ ]:
from os.path import abspath, dirname, join, normpath, relpath
from matplotlib.pyplot import figure
import scipy.interpolate as interp
import matplotlib.pyplot as plt
from datetime import datetime
from dolfyn.adp import api
import xarray as xr
import pandas as pd
import numpy as np
import scipy.io
import netCDF4
import math 
import utm
# MHKiT Imports
from mhkit.dolfyn.rotate import api as ap
from mhkit import dolfyn as dlfn
from mhkit.river.io import d3d 
from mhkit import river

## Preparing the Field Data

Field data was collected in 2010. As can be seen below 2 passes were collected across the river. In the next section we will import this data into MHKiT and then interpolate these two passes into a single idealized transect for comparison with the simulated Deflt3D data.

<img src="TRTS_transect_map.png" width=720 height=640 />

### Importing ADCP Data

The MHKiT DOLfYN api module can import .PDO and other ADCP and ADV binary formats to an xarray. For this analysis we import two transect passes stored in the 'data' folder in .PDO format. For comparison to the numerical model we want to average the two transects into one xarray ('ds_02_03').

In [ ]:
# Read in the two transect passes
ds_02 = api.read('data/tanana_transects_08_10_10_0_002_10-08-10_142214.PD0')
ds_03 = api.read('data/tanana_transects_08_10_10_0_003_10-08-10_143335.PD0')
# Create one dataset from the two passes
ds_02_03= xr.merge([ds_02,ds_03])
# Print the xarray data
ds_02_03

### Convert the the coordinate system to UTM

The ADCP location data is stored in longitude and latitude coordinates by default. However, latitude and longitude cannot be directly converted to a distance therefor here we convert from latitude and longitude to $UTM_x$ and $UTM_y$. This gives us the ability to calculate the linear line of best fit for the idealized transect.

In [ ]:
data=utm.from_latlon(ds_02_03.latitude_gps, ds_02_03.longitude_gps, 6, 'W') # combined transects 
latitude=data[0]
longitude=data[1]

Calculate ideal linear transect using combined ds_02_03 data set. Using the numpy polyfit function to find the slop and intercept we can then use those to recalculate the UTM y coordinated to be the ideal linear transect.

In [ ]:
# Linear regression using first order polyfit
a,b = np.polyfit(latitude, longitude,1)

# Initialize the figure
figure(figsize=(8,6))

# Plot the original transect data for comparison
transect_1 = utm.from_latlon(ds_02.latitude_gps, ds_02.longitude_gps, 6, 'W') 
transect_2=utm.from_latlon(ds_03.latitude_gps, ds_03.longitude_gps, 6, 'W') 

latitude_1=transect_1[0]
longitude_1=transect_1[1]

latitude_2=transect_2[0]
longitude_2=transect_2[1]

plt.plot(latitude_1,longitude_1, label= 'GPS Transect 1')
plt.plot(latitude_2,longitude_2, label= 'GPS Transect 2')

# Plot the Idealized Transect
plt.plot(latitude,a*latitude+b, label= 'Linear Regression')
plt.legend()
plt.xlabel('UTM x (m)')
plt.ylabel('UTM y (m)')

### Get GPS & Ideal point

Why are we doing this here? We do not seem to use this data.

In [ ]:
gps = np.array([
    [lat, lon] for lat, lon in zip(latitude, longitude)
    ]) 
gps_points = pd.DataFrame(gps, columns= ['latitude','longitude'])

ideal= np.array([ 
    [lat, lon] for lat, lon in zip(latitude, a*latitude+b)
    ]) 
ideal_sampled_points = pd.DataFrame(ideal, columns= ['latitude','longitude'])

### Range offset

What is range offset?

In [ ]:
# What is this doing?
api.clean.set_range_offset(ds_02_03,0)
# why is this being shown?
ds_02_03.range

### Correlation filter 

Data is fileted for a minimum colleation of 40 counts 

Why are we showing this to the reader?

In [ ]:
# You plot first
ds_02_03.corr.sel(beam=1, range=slice(0,10)).plot()
# then rotate the data...is this important to do after the plot?
api.rotate2(ds_02_03, 'earth', inplace=True)
ds_02_03 = api.clean.correlation_filter(ds_02_03, thresh=40)

### Declination 

If your declination is not already set you can correct for declination with the MHKIT DOLfYN ap's set_declination. If it was already been set with the deployment software it will be saved under the attributes  `magnetic_var_deg` 
> Is this only for Teledyn or always under this variable?

In [ ]:
ap.set_declination(ds_02_03, 15.7, inplace=True) # 15.7 deg East for Nenana Alaska 

### Bottom Filter 
MHKiT DOLfYN has the functionality to find the river bottom based on the along-beam acoustic amplitude data recorded from the ADCP. However, if depth sounder data is available it can be more reliable at representing the river bottom. Here we use the depth sounder `single_beam` to create an array that can be used as a filter when multiplied with the velocity data.

In [ ]:
# Explain inline what is happening
single_beam = ds_02_03.where(ds_02_03.dist_bt > 0 )
bottom=np.min(single_beam.dist_bt,axis=0)
bottom_avg= interp.griddata(gps_points,bottom,ideal_sampled_points, method='linear')
bottom_filter = d3d.create_points(x=bottom_avg, y=ds_02_03.range.to_numpy(), waterdepth=1)
river_bottom_filter= []

for index, row in  bottom_filter.iterrows():
    if row['x'] > row['y']: 
        filter = 1    
    else: 
        filter= float("nan")
    river_bottom_filter= np.append(river_bottom_filter, filter)

### Raw Points

Why are we getting raw point?

In [ ]:
# what are these tiles doing?
lat= np.tile(latitude, np.size( ds_02_03.range))
long= np.tile(longitude, np.size( ds_02_03.range))
depth =np.repeat( ds_02_03.range,np.size(latitude))
Point={'latitude': lat, 'longitude': long, 'waterdepth': depth}
points=pd.DataFrame(Point)

# If we are multiplying by river_bottom_filter are these truely raw?
points['north']= np.ravel(ds_02_03.vel[1, :,:])* river_bottom_filter
points['east']= np.ravel(ds_02_03.vel[0, :,:])* river_bottom_filter
  # You call it depth above... Maybe keep the same name?
points['vert']= np.ravel(ds_02_03.vel[2, :,:])* river_bottom_filter
points= points.dropna()
# Show points
points

### Ideal Transect 

Why is this all the way up down here away from the work relevant to it above?

In [ ]:
lat_points = np.tile(latitude, np.size(ds_02_03.range))
long_points = np.tile(a*latitude+b, np.size(ds_02_03.range))
depth_points = np.repeat( ds_02_03.range, np.size(latitude))

ADCP_points={
    'latitude': lat_points, 
    'longitude': long_points, 
    'waterdepth': depth_points
    }
ADCP_points=pd.DataFrame(ADCP_points)
ADCP_points

### Ideal Transect Downsampled

Discuss why we are down sampling

In [ ]:
# Interpolate points by getting min & max first
stat_lat = min(ADCP_points.latitude)
start_long= min(ADCP_points.longitude)

end_lat= max(ADCP_points.latitude)
end_long= min(ADCP_points.longitude)

# Why are we printing this?
(end_lat-stat_lat)/(math.cos(math.atan(a))*15)

In [ ]:
lat_downsampeled = np.linspace(stat_lat,end_lat,10)
long_downsampeled= a*lat_downsampeled+b

In [ ]:
lat_points_downsampled= np.tile(lat_downsampeled, np.size( ds_02_03.range))
long_points_downsampled= np.tile(long_downsampeled, np.size( ds_02_03.range))
depth_points_downsampled =np.repeat( ds_02_03.range,np.size(lat_downsampeled))

ADCP_points_downsamples={'latitude': lat_points_downsampled, 'longitude': long_points_downsampled, 'waterdepth': depth_points_downsampled}
ADCP_points_downsamples=pd.DataFrame(ADCP_points_downsamples)
ADCP_points_downsamples

# Interpolating Velocities for Ideal Transect 

In [ ]:
# Project velocity onto ideal tansect 
ADCP= pd.DataFrame()
ADCP['East']=  interp.griddata(points[['latitude','longitude','waterdepth']],points['east'],ADCP_points, method='linear', fill_value=0)
ADCP['North']= interp.griddata(points[['latitude','longitude','waterdepth']],points['north'],ADCP_points, method='linear', fill_value=0)
ADCP['Vertical']=  interp.griddata(points[['latitude','longitude','waterdepth']],points['vert'],ADCP_points, method='linear', fill_value=0)
ADCP['Magnitude']= np.sqrt(ADCP.East**2+ADCP.North**2+ADCP.Vertical**2)
ADCP

In [ ]:
max_plot=3
min_plot=0

# Plotting 
plt.figure(figsize=(10,4.4))
contour_plot = plt.tripcolor(
    ADCP_points.latitude, 
    -ADCP_points.waterdepth, 
     ADCP.Magnitude*river_bottom_filter,
    vmin=min_plot,
    vmax=max_plot
)

plt.xlabel('UTM x (m)')
plt.ylabel('Water Depth (m)')
cbar= plt.colorbar(contour_plot)
cbar.set_label('velocity [m/s]')
plt.ylim([-8.5,-1])
plt.xlim([400950,401090])
plt.plot(ideal_sampled_points.latitude,-bottom_avg,'k', label= 'river bottom')
plt.legend(loc= 7)

# Interpolating Downsampled Velocites for ideal transect 

In [ ]:
lat_downsampeled = np.linspace(stat_lat,end_lat,10)
long_downsampeled= a*lat_downsampeled+b

ideal_downsampeled= np.array([ [lat, long] for lat, long in zip(lat_downsampeled,
                            a*lat_downsampeled+b)]) 
ideal_sampled_points_downsampled = pd.DataFrame(ideal_downsampeled, columns= ['latitude','longitude'])

bottom_avg_downsampled= interp.griddata(gps_points, bottom, ideal_sampled_points_downsampled, method='linear')
bottom_filter_downsampled = d3d.create_points(x=bottom_avg_downsampled, y=ds_02_03.range.to_numpy(), waterdepth=1)
river_bottom_filter_downsampled= []

for index, row in  bottom_filter_downsampled.iterrows():
    if row['x'] > row['y']: 
        filter= 1
    
    else: 
        filter= float("nan")
    river_bottom_filter_downsampled= np.append(river_bottom_filter_downsampled, filter)

In [ ]:
ADCP_downsamples= pd.DataFrame()

ADCP_downsamples['East']=  interp.griddata(points[['latitude','longitude','waterdepth']],points['east'],ADCP_points_downsamples, method='linear', fill_value=0)
ADCP_downsamples['North']= interp.griddata(points[['latitude','longitude','waterdepth']],points['north'],ADCP_points_downsamples, method='linear', fill_value=0)
ADCP_downsamples['Vertical']=  interp.griddata(points[['latitude','longitude','waterdepth']],points['vert'],ADCP_points_downsamples, method='linear', fill_value=0)
ADCP_downsamples['Magnitude']= np.sqrt(ADCP_downsamples.East**2+ADCP_downsamples.North**2+ADCP_downsamples.Vertical**2)
ADCP_downsamples

In [ ]:
max_plot=3
min_plot=0

# Plotting 
plt.figure(figsize=(10,4.4))
contour_plot = plt.tripcolor(
    ADCP_points_downsamples.latitude, 
    -ADCP_points_downsamples.waterdepth, 
     ADCP_downsamples.Magnitude*river_bottom_filter_downsampled,
    vmin=min_plot,
    vmax=max_plot
)

plt.xlabel('UTM x (m)')
plt.ylabel('Water Depth (m)')
cbar= plt.colorbar(contour_plot)
cbar.set_label('velocity [m/s]')
plt.ylim([-8.5,-1])
plt.xlim([400950,401090])
plt.plot(ideal_sampled_points.latitude,-bottom_avg,'k', label= 'river bottom')
plt.legend(loc= 7)

# USGS Water Level

In [ ]:
# Use the requests method to obtain 10 years of daily discharge data
data = river.io.usgs.request_usgs_data(station="15515500",
                            parameter='00065',
                            start_date='2010-08-10',
                            end_date='2010-08-10',
                            data_type='Instantaneous')

# Print data
print(data)

 # Delft3D data

In [ ]:
# Simulated data
# Downloading Data
d3d_data = netCDF4.Dataset('data/tanana81010_final_map.nc')

In [ ]:
# Printing variable and description
for var in d3d_data.variables.keys():
    try: 
        d3d_data[var].long_name
    except:
        print(f'"{var}"')        
    else:
        print(f'"{var}": {d3d_data[var].long_name}')

In [ ]:
ADCP_points_downsamples_xy= ADCP_points_downsamples.rename(columns={"latitude": "x", "longitude": "y"})


In [ ]:
variables= ['ucy', 'ucx','ucz']
D3d= d3d.variable_interpolation(d3d_data, variables, points= ADCP_points_downsamples_xy)

In [ ]:
D3d['Magnitude']=  np.sqrt(D3d.ucy**2+D3d.ucx**2+D3d.ucz**2)

In [ ]:
max_plot=3
min_plot=0

# Plotting 
plt.figure(figsize=(10,4.4))
contour_plot = plt.tripcolor(
    D3d.x, 
    -D3d.waterdepth, 
    D3d.Magnitude*river_bottom_filter_downsampled,
    vmin=min_plot,
    vmax=max_plot,
    #shading='gouraud'
    alpha=1
)
#*river_bottom_filter_downsampled

plt.xlabel('UTM x (m)')
plt.ylabel('Water Depth (m)')
#plt.title('Velocity in the Y direction')
cbar= plt.colorbar(contour_plot)
cbar.set_label('velocity [m/s]')
plt.ylim([-8.5,-1])
plt.xlim([400960,401090])
plt.plot(ideal_sampled_points.latitude,-bottom_avg,'k', label= 'river bottom')
plt.legend(loc= 7)

# L1 Error Calulations

In [ ]:
# L1

#north 
L1_Magnitude= abs(ADCP_downsamples.Magnitude-D3d.Magnitude)/ADCP_downsamples.Magnitude


In [ ]:
bottom_filter_downsampled= []
for i in  L1_Magnitude:
    if 1 > i: 
        filter= 1
    
    else: 
        filter= float("nan")
    bottom_filter_downsampled= np.append(bottom_filter_downsampled, filter)

In [ ]:
max_plot=1
min_plot=0
# Plotting 
plt.figure(figsize=(10,4.4))
contour_plot_L1 = plt.tripcolor(
    D3d.x, 
    -D3d.waterdepth, 
    L1_Magnitude*bottom_filter_downsampled*river_bottom_filter_downsampled,
        vmin=min_plot,
    vmax=max_plot
)

plt.xlim([400960,401090])
plt.ylim([-8.5,-1])
plt.xlabel('UTM x (m)')
plt.ylabel('Water Depth (m)')
#plt.title('L1 error between Delft3D and ADCP data for North velocity')
cbar= plt.colorbar(contour_plot_L1)
cbar.set_label('L1 velocity error')
plt.plot(ideal_sampled_points.latitude,-bottom_avg,'k', label= 'river bottom')
plt.legend(loc= 7)

MAE= np.sum(L1_Magnitude*bottom_filter_downsampled*river_bottom_filter_downsampled)/len(L1_Magnitude[L1_Magnitude< 1000 ])
MAE

# L2 Error Calculations

In [ ]:
# L2 

L2_Magnitude= ((ADCP_downsamples.Magnitude-D3d.Magnitude)/ADCP_downsamples.Magnitude)**2

MSE=np.sum(L2_Magnitude*bottom_filter_downsampled*river_bottom_filter_downsampled)/np.size(L2_Magnitude[L2_Magnitude< 1000])

print(MSE)
L2_Mag=L2_Magnitude[L2_Magnitude>1]= float('nan')

max_plot=1
min_plot=0
# Plotting 
plt.figure(figsize=(10,4.4))
contour_plot_L2 = plt.tripcolor(
    D3d.x, 
    -D3d.waterdepth, 
    L2_Magnitude*bottom_filter_downsampled*river_bottom_filter_downsampled,
    vmin=min_plot,
    vmax=max_plot
)

plt.xlim([400960,401090])
plt.ylim([-8.5,-1])
plt.xlabel('UTM x (m)')
plt.ylabel('Water Depth (m)')
#plt.title('L2 error between Delft3D and ADCP data for North velocity')
cbar= plt.colorbar(contour_plot_L1)
cbar.set_label('L2 velocity error')
plt.plot(ideal_sampled_points.latitude,-bottom_avg,'k', label= 'river bottom')
plt.legend(loc= 7)

In [ ]:
# L inf

L_inf=np.nanmax(L1_Magnitude[L1_Magnitude< 1000 ])
L_inf